<a href="https://colab.research.google.com/github/d-zg/Poker-Action-Classifier/blob/main/2_group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install mrjob
!pip install mrjob
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 KB 17.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 18.5 MB/s eta 0:00:00


# Using multi steps

A MR job has multiple steps. Read about reducer and steps in https://buildmedia.readthedocs.org/media/pdf/mrjob/latest/mrjob.pdf

A “step” consists of a mapper, a combiner, and a reducer. All of those are optional, though you must have at least one.
So you could have a step that’s just a mapper, or just a combiner and a reducer.
When you only have one step, all you have to do is write methods called mapper(), combiner(), and
reducer()

In [5]:
%%file 2_group.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import json

'''


See http://mike.teczno.com/notes/streaming-data-from-twitter.html for parsing info.
Get the screen name by accessing tweet['user']['screen_name']

Output:
number_of_most_tweets    username

(note that MRJob automatically adds a tab for you)
'''
# read the 
class GroupMaxTweets(MRJob):
    # The _ means the field does not matter.;
    def mapper(self, _, line):
        try:
          tweet = json.loads(line)
          yield tweet['user']['screen_name'], 1
            # do something
        except:
            pass

    def reducer(self, key, counts):
        # yield something -- hint you can yield a tuple
        yield None, (key, sum(counts))
        pass
   
    def reducer_max(self, key, counts):
        # yield something
        largest = 0
        name = None
        for count in counts:
          if count[1] > largest:
            largest = count[1]
            name = count[0]
        yield largest, name
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper, reducer=self.reducer),
            MRStep(reducer=self.reducer_max)
        ]


if __name__ == '__main__':
    GroupMaxTweets.run()

Overwriting 2_group.py


## Step 1. Run MR program locally(Colab)

In [6]:
%%time
!python 2_group.py -r local nashville-tweets-2019-01-28

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/2_group.root.20230330.215543.093364
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/2_group.root.20230330.215543.093364/output
Streaming final output from /tmp/2_group.root.20230330.215543.093364/output...
88	"rpsabo"
Removing temp directory /tmp/2_group.root.20230330.215543.093364...
CPU times: user 41.2 ms, sys: 21.8 ms, total: 63 ms
Wall time: 4.78 s


## Step 2. Run MR program on AWS EMR

### Please copy nashville-tweets-2019-01-28 into your S3 bucket.

In [ ]:
%%time
!python 2_group.py -r emr s3://<s3 url of the Tweet dataset> --cloud-tmp-dir=s3://<s3 url of tmp directory> --cluster-id=<cluster ID> --conf-path <mrjob.conf file path in colab> > 2_group.out

In [ ]:
#test the code run

f = open('2_group.out')
lines = f.readlines()
f.close()

assert lines[0][:-1] in ['88\t"rpsabo"']
print("test passed")